## 1. Data Cleaning

We follow the __Quartz: Bad Data Guide__ at https://github.com/Quartz/bad-data-guide, and conduct the following steps:

1. Check data size before reading-in. As a rule of thumb, the data size should be at most 1/4 of the computer's RAM.
1. Summarize the variables through grouping. Understand the meaning of each variable, and group them into meaningful baskets.
1. Inspect missing values. Analyze their causes, and decides what to do with them.
1. Variable transformation. 
    - Properly format variables. For dates, convert them into computable formats; for texts, remove the redundencies and check for spelling; for numerical values, make sure the units are consistent. 
    - Expand the scope of data. Can we include more variables that might be of interest?




We first do each step manually, then wrap our actions into functions so that the process is more replicable on new datasets. We export the cleaned training and test set.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
sns.set()

### 1.1 Data Size & First Peek

In [2]:
!ls -lh ../data

total 33M
drwxr-xr-x 1 nleea 197609    0 Dec  7 21:54 census_data
-rw-r--r-- 1 nleea 197609 4.0M Dec  7 21:55 census_data.zip
-rw-r--r-- 1 nleea 197609 336K Dec  3 18:33 LendingClubData_new_training.xlsx
-rw-r--r-- 1 nleea 197609 1.5M Nov  7 18:52 LendingClubData_testing.xlsx
-rw-r--r-- 1 nleea 197609  16M Dec  2 14:45 LendingClubData_training.xlsx
drwxr-xr-x 1 nleea 197609    0 Dec  3 02:59 statewide_monthly_unemployment
-rw-r--r-- 1 nleea 197609 1.1M Dec  7 14:44 test_cleaned.xlsx
-rw-r--r-- 1 nleea 197609  11M Dec  7 14:44 training_cleaned.xlsx
-rw-r--r-- 1 nleea 197609  25K Dec  3 03:01 unemployment.csv
-rw-r--r-- 1 nleea 197609 294K Dec  3 08:55 us_company_names_industries.xlsx
-rw-r--r-- 1 nleea 197609  849 Dec  3 17:10 us_quarterly_gdp_growth.csv


The raw data is about 16MB, safe to read into a RAM of 8GB.  

In [3]:
%%time
## Reading in the dataset
rawTraining = pd.read_excel("../data/LendingClubData_training.xlsx")
rawTest = pd.read_excel("../data/LendingClubData_testing.xlsx")

Wall time: 31.2 s


The training set contains 35808 rows, and the test set 3978 rows; both contain the same set of 145 features. Our training to test sample size is very close to 9:1.

In [4]:
rawTraining.shape
# rawTraining.head(5)
rawTest.shape
# rawTest.head(5)
rawTraining.shape[0] / rawTest.shape[0]

(35808, 145)

(3978, 145)

9.001508295625943

A quick peek shows that out of the 145 features, 83 are completely empty. Though they apparently cannot be used for predictive purpose in this iteration, their future inclusion may improve the model's quality. Before hastily discarding them, we want to assess whether they can add any _material insights_ to the current dataset. To do this, we have to first understand what the non-empty variables are telling us.

In [5]:
# The function takes in a lending club dataframe and returns a list of names of columns that are empty.
def empty_columns(lendingClub_df):
    empty_col_bool = lendingClub_df.isnull().sum(axis=0) == lendingClub_df.shape[0]
    empty_cols = lendingClub_df.columns[empty_col_bool]
    return empty_cols
# Do training and test share the same empty columns? Yes!
(empty_columns(rawTraining) != empty_columns(rawTest)).sum()
# We can use one variable to reflect the empty columns in both. We use underline separated names for functions, and upper-case letters
# for variable names.
emptyColumns = empty_columns(rawTraining)
emptyColumns.unique().shape

def non_empty_columns(lendingClub_df):
    nonEmptyCol_bool = lendingClub_df.isnull().sum(axis=0) != lendingClub_df.shape[0]
    nonEmptyCols = lendingClub_df.columns[nonEmptyCol_bool]
    return nonEmptyCols
nonEmptyColumns = non_empty_columns(rawTraining)
nonEmptyColumns.shape

0

(83,)

(62,)

### 1.2 Summary of Variables

__Cleaning the Data Dictionary__

To conviniently consult the data dictionary, we read it in as a pandas DataFrame. 

- The original dictionary was split between three excel sheets. We manually copied and pasted three sheets into one, and call it the "flattened" dictionary.

- Notice that some variable names in the data dictionary contain a trailing space. We remove them.

- We further discover there's one column that isn't documented in the data dictionary -- `verification_status_joint`. A closer look shows that in the data dictionary, the same variable is termed as `verified_status_joint`. We'd modify the data dictionary to ensure consistency in naming. 

In [6]:
## Read in the data dictionary. 
dataDict = pd.read_excel("../dict/LendingClubDataDictionary_Flattened.xlsx")
dataDict.columns = ["Variable", "Description"]
dataDict.drop_duplicates(subset="Variable", keep="first", inplace=True)
# Notice some variable names in the data dictionary end with an extra space. We need to remove them.
dataDict["Variable"] = dataDict["Variable"].str.rstrip()
variableNames = list(dataDict["Variable"])
# Any columns not documented in the data dictionary?
columns = list(rawTraining.columns)
inDictBool = [(i in variableNames) for i in columns]
(~np.array(inDictBool)).sum()
# We change the naming in data dictionary to ensure consistency
dataDict["Variable"] = dataDict["Variable"].str.replace("verified_status_joint","verification_status_joint")
# Set variable as index for easier selection.
dataDict.set_index("Variable", inplace=True)
dataDict.to_excel("../dict/LendingClubDataDictionary_Cleaned.xlsx")

1

__Summarizing the non-empty variables__

We have 62 non-empty variables. To better summarize the content, we put them into four baskets:

- Loan Conditions:
    - _Information on the application_. Such as `Loan Amount` applied for, `Title`, `Purpose`, `Description` for the loan and `Application Type` indicating whether the application is made by an individual or jointly), 
    - _Loan parameters_. Such as the actual `Amount Funded`, `Interest Rate`, and `Grade` assigned to the loan
    - _`Loan Status`_. Whether it's fully paid for charged-off.
- Borrower's Financial Conditions:
    - _Employment Status_. `Annual Income`, `Employer Title`, `Employment Length` all fall under this category.
    - _Residental Status_. This includes `Homeonwership`, `State`, `Zip Code`.
    - _`Debt-to-income Ratio`_ is also in this category.
- Borrower's Credit Situation:
    - _Length of credit history_, as reflected in `Earliest Credit Line`.
    - _History of late payment_, as captured by `Delinquencies in 2 years`, `Monthes Since Last Delinquency`, number of `Accounts Now Delinquent`, `Delinquent Amount`, `Chargeoffs in 12 Months` etc.
    - _Credit utilization_. This includes `Revolving Balance`, `Revolving Balance Utilization`, `Number of Total Accounts`. 
    - _New credit line inquiries_. `Inquiries in 6 months`, `Last Credit Pulled Date`.
    - _Other credit burdens_ such as number of `Tax Liens`, number of `Public Bankcruptcy Records` or `Derogatory Public Records`, `Collections in 12 Months`.
- Payment on this loan.
    - _Payments received_ so far on principle and interest.
    - _Settlement plan_. If default occurs, whether a `Debt Settlement Flag`(plan) is agreed upon, and parameters and progress on that settlement.

In [7]:
# Tidying up the variable names. 
variable_name_original = list(non_empty_columns(rawTraining))
variable_name_tidy = ["Loan Amount", "Funded Amount", "Funded Amount Investor", "Term", "Interest Rate", "Installment", "Grade", 
                      "Sub Grade", "Employer Title", "Employment Length", "Home Ownership", "Annual Income", "Verification Status",
                     "Issued Date", "Loan Status", "Payment Plan", "Description", "Purpose", "Title", "Zip Code", "State", "Debt-to-income Ratio",
                     "Delinquencies in 2 years", "Earliest Credit Line", "Inquiries in 6 months", "Months Since Last Delinquency",
                     "Months Since Last Public Record", "Open Accounts", "Derogatory Public Records", "Revolving Balance", 
                     "Revolving Balance Utilization", "Number of Total Accounts", "Initial List Status", "Outstanding Principle", 
                     "Outstanding Principle Investor", "Total Payment", "Total Payment Investor", "Total Received Principle", "Total Received Interest",
                     "Total Received Late Fee", "Recoveries", "Collection Recovery Fee", "Last Payment Date", "Last Payment Amount", "Last Credit Pulled Date",
                     "Collections in 12 Months", "Policy Code", "Application Type", "Accounts Now Delinquent", "Chargeoffs in 12 Months", "Delinquent Amount", 
                      "Public Bankcruptcy Records", "Tax Liens", "Hardship Flag", "Disbursement Method", "Debt Settlement Flag", "Debt Settlement Flag Date",
                     "Settlement Status", "Settlement Date", "Settlement Amount", "Settlement Percentage", "Settlement Term"]
tidy_original_dict = dict(zip(variable_name_tidy, variable_name_original))

variable_names_grouped = {
    "Loan Condition": ["Loan Amount", "Funded Amount", "Funded Amount Investor", "Term", "Interest Rate", "Installment", "Grade", 
                      "Sub Grade", "Loan Status", "Issued Date","Title", "Description", "Purpose", "Application Type", "Policy Code",
                      "Initial List Status"],
    "Borrower Financial Condition": ["Employer Title", "Employment Length", "Home Ownership", "Annual Income", "Verification Status",
                     "Zip Code", "State", "Debt-to-income Ratio"],
    "Credit Situation": ["Delinquencies in 2 years", "Earliest Credit Line", "Inquiries in 6 months", "Months Since Last Delinquency",
                     "Months Since Last Public Record", "Open Accounts", "Derogatory Public Records", "Revolving Balance", 
                     "Revolving Balance Utilization", "Number of Total Accounts", "Last Credit Pulled Date", "Collections in 12 Months", 
                        "Accounts Now Delinquent", "Chargeoffs in 12 Months", "Delinquent Amount", "Public Bankcruptcy Records", "Tax Liens",
                        ],
    "Payment on Loan": ["Outstanding Principle", 
                     "Outstanding Principle Investor", "Total Payment", "Total Payment Investor", "Total Received Principle", 
                        "Total Received Interest","Total Received Late Fee", "Recoveries", "Collection Recovery Fee", "Payment Plan",
                        "Last Payment Date", "Last Payment Amount", "Hardship Flag", "Disbursement Method", "Debt Settlement Flag", 
                        "Debt Settlement Flag Date", "Settlement Status", "Settlement Date", "Settlement Amount", "Settlement Percentage", "Settlement Term"]
}
variable_names_grouped_list = []
for group in variable_names_grouped.values():
    variable_names_grouped_list.extend(group)
len(variable_names_grouped_list)

62

### 1.3 Missing Values

#### Empty Variables

The empty variables fall into one of the three categories: information on joint application, borrower's other credit burdens, and hardship plan status. We think that while both information on co-applicants and hardship status can reveal interesting patterns, only other credit burdens can add material insights to our analysis of default probability.

- Information on __joint applicants__. Our dataset only contains information on loans applied for by individuals. However, we don't think that joint application should be treated in a fundamentally different way than an individual ones, as long as we can properly aggregate information on all applicants. In future iterations where joint applications are present, the data should be processed in a similar way to this project.
- Applicant's __other credit burdens__. This includes information on the applicant's installment accounts, such as mortgage, auto-loan, and sstudent loans.<span style="color:red"> We think this information can be very helpful for predicting default in some cases. </span> Installment loans typically are collateralized (with the exception of student loans), while Lending Club loans are mostly unsecured, so defaulting on the former usually has a much severer consequence than the latter. If one has already recently defaulted on a mortgage or auto loan, he/she is very likely under significant hardship, and will default on a lending club loan as well. _As such, we recommend including this information in future data collection_.
- __Hardship plan status__. Lending Club offers borrowers three-month "hardship" plans when only a reduced installment has to be paid. Analysis on hardship plan data can shed light on some interesting questions, such as whether enrollment in the hardship plan signals stronger willingness to avoid default, should Lending Club make automatic recommendation of hardship plans to all borrower's likely to become delinquent, etc. 

We remove the empty columns before proceeding to further analysis.

In [8]:
# Check the meaning of empty variables.
dataDict.loc[list(emptyColumns), :]

,Description
Variable,
id,A unique LC assigned ID for the loan listing.
member_id,A unique LC assigned Id for the borrower member.
url,URL for the LC page with listing data.
next_pymnt_d,Next scheduled payment date
mths_since_last_major_derog,Months since most recent 90-day or worse rating
...,...
hardship_dpd,Account days past due as of the hardship plan ...
hardship_loan_status,Loan Status as of the hardship plan start date
orig_projected_additional_accrued_interest,The original projected additional interest amo...


In [9]:
# Removing the emtpy columns, and re-order the columns according to grouping.
def get_non_empty_columns(lendingClub_df_raw):
    training_nonempty = lendingClub_df_raw[list(non_empty_columns(rawTraining))]
    training_nonempty.columns = variable_name_tidy
    training_nonempty = training_nonempty.reindex(columns=variable_names_grouped_list)
    return training_nonempty

training_nonempty = rawTraining.pipe(get_non_empty_columns)
test_nonempty = rawTest.pipe(get_non_empty_columns)


#### Missing Values in Columns

There are 19 columns that contain missing values. 

- Some are discretionary fields that the __applicant didn't fill in__. These include loan `Title`, `Description` for loan purpose, `Employer Title` and `Employment Length`. As the fact that applicant opted to omit these might contain meaningful information, we don't discard them. We simply replace them with "".
- Some are due to __inconsistency in data recording__. 
    - For applicants who have no delinquencies in the last two years, `Months Since Last Delinquency` are recorded as 0 for some, and left as blank for most. We fill the blank entries with 0, keeping in mind that 0 means no delinquency record. For `Months Since Last Public Record`, we repeat the same procedure against the number of `Derogatory Public Records`.
    - All loans that have no `Last Payment Date` have been defaulted on. A closer look reveals that no regular payment was ever recieved on these loans. To ensure consitency in computation for days since the last missed payment, we fill these blanks with loan `Issued Date`.
- Some seem to be caused by __inadequecy in data collection__. `Chargeoffs in 12 Months`, `Collections in 12 Months` have values of either 0 or blank. `Revolving Balance Utilization`, `Public Bankcruptcy Records`, `Last Credit Pulled Date` and `Tax Liens` contain both 0 and blank. As we don't know whether the blank values indicate no such incidences occur or no data available, we discard them in this iteration.<span style="color:orange"> A total of 749 data points (about 2% of the training set) are removed from the training set; 1 data point is removed from the test set. </span> <span style="color:red"> We think that in future iteration, such inconsistency can be resolved through better exception handling on when referencing external databases </span> 
- At last, some are empty because the __features don't apply to the entry__. Most loans don't have a settlement plan, but for those that do, how the settlement is formualated might shed insights on how lending club might capture loss. We leave them as is.

In [10]:
# Check what non-empty columns contain missing values
missing_value_by_column = training_nonempty.isnull().sum(axis=0).to_frame()
missing_value_nonzero = missing_value_by_column[(missing_value_by_column != 0).any(axis=1)]
missing_value_nonzero.shape
missing_value_nonzero

(19, 1)

,0
Title,12
Description,11188
Employer Title,2220
Employment Length,953
Months Since Last Delinquency,22933
Months Since Last Public Record,33140
Revolving Balance Utilization,49
Last Credit Pulled Date,2
Collections in 12 Months,56
Chargeoffs in 12 Months,56


In [11]:
# Replace missing values in Title, Description, Employer Title and Employment Length with "Not Provided"
def replace_with_empty_string(lendingClub_df):
    variable_list = ['Title', 'Description', 'Employer Title', 'Employment Length']
    for i in variable_list:
        lendingClub_df[i].fillna("", inplace=True)
    return lendingClub_df

# For Months Since Last Delinquency, and Months Since Last Public Record, we fill the banks with 0,
# keeping in mind 0 means no delinquency/public records.
def fill_blank_months_with_zero(lendingClub_df):
    variable_list = ["Months Since Last Delinquency", "Months Since Last Public Record"]
    for i in variable_list:
        lendingClub_df[i].fillna(0, inplace=True)
    return lendingClub_df

# Fill Last Payment Date of loans on which no payments were ever made with the loan issuance date.
def replace_empty_last_pmt_d_with_issuance_d(lendingClub_df):
    no_last_pmt_bool = lendingClub_df["Last Payment Date"].isnull()
    (lendingClub_df["Last Payment Date"])[no_last_pmt_bool] = (lendingClub_df["Issued Date"])[no_last_pmt_bool]
    return lendingClub_df


# Discard the missing entries in Revolving Balance Utilization, Collections in 12 Months, Chargeoffs in 12 Months, 
# Public Bankcruptcy Records, Tax Liens and Last Credit Pulled Date. 749 entries are removed in the process.
def drop_no_data_entries(lendingClub_df):
    variable_list = ["Revolving Balance Utilization", "Collections in 12 Months", "Chargeoffs in 12 Months",
                    "Public Bankcruptcy Records", "Tax Liens", "Last Credit Pulled Date"]
    lendingClub_df.dropna(axis=0, how='any', subset=variable_list, inplace=True)
    return lendingClub_df

def missing_value_handling(lendingClub_df):
    lendingClub_df.pipe(replace_with_empty_string).pipe(fill_blank_months_with_zero).pipe(replace_empty_last_pmt_d_with_issuance_d).pipe(drop_no_data_entries);
    return lendingClub_df

# training_nonempty.pipe(missing_value_handling)
# test_nonempty.pipe(missing_value_handling)

### 1.4 Variable Transformation

#### Removing Columns With No Variation

13 columns contain only one value. 

- Some variables contain only one value because the dataset was pre-processed (possibly by our instructor) to simplify the problem. Examples include `Application Type`, `Hardship Flag`, and `Payment Plan`. `Outstanding Principle` is uniformly zero because in our dataset, a loan is either charged-off or fully paid, in either case no more princple payment is expected. 
- Some might result from incomplete record. It's hard to believe that out of more than thirty thousand applicants, no one has an `Account Now Delinquent`. The same goes for `Tax Liens`, `Chargeoffs in 12 Months` and `Collection in 12 Months`. <span style=color:red> We recommend looking into the data collection process to check for completeness of data. </span>

In [12]:
# Check which columns in the dataset contain no variation. 
def columns_no_variation(lendingClub_df):
    col_names_list = list(lendingClub_df.columns)
    num_of_features = len(col_names_list)
    col_names_no_variation = []
    for i in range(num_of_features):
        if lendingClub_df.iloc[:, i].unique().size == 1:
            col_names_no_variation.append(col_names_list[i])
    return col_names_no_variation

def drop_columns_no_variation(lendingClub_df):
    lendingClub_df.drop(columns = columns_no_variation(lendingClub_df), inplace=True)
    return lendingClub_df

# # Verify that columns without variation in the training and test set are the same.
# columns_no_variation(training_nonempty) == columns_no_variation(test_nonempty)
# training_nonempty.pipe(drop_columns_no_variation)
# test_nonempty.pipe(drop_columns_no_variation)


#### Formatting the Variables

We did the following basic manipulations:

- `Loan Status` was changed to indicators.
- `Description`: removed strings that are not meaningful.
- `Employment Length`: we checked sorting works on this column.
- `Earliest Credit Line`: we added a column that represents the length of credit history at the time of application in months.
- `Public Bankcruptcy Records`: we discovered that no `Derogatory Public Records` entry was less than `Public Bankcruptcy Records`, which very likely implies that Derogatory Records include Bankcruptcy records by definition. 
- `Total Received Principle`: Some terms are 0.00001 away from their closest integer value. Round them.
 

In [13]:
# Use a chargeoff indicator for Loan Status.
def chargeoff_indicator(lendingClub_df):
    defaultDict = {"Charged Off" : 1, "Fully Paid" : 0}
    lendingClub_df["Loan Status"].replace(defaultDict, inplace=True)
    return lendingClub_df

# training_nonempty.pipe(chargeoff_indicator)
# test_nonempty.pipe(chargeoff_indicator)

In [14]:
# Remove the formatting strings from description.
def format_description(lendingClub_df):
    lendingClub_df["Description"] = lendingClub_df["Description"].str.replace(r"Borrower.* > ", "").str.replace(r"<.*>", "").str.strip().astype(str)
    return lendingClub_df

# # Check if sorting works on Employment Length. It does!
# training_nonempty["Employment Length"].sort_values(ascending=False)

# Add a column that represent the length of credit history at the time of issuance in months.
def add_credit_history_in_months(lendingClub_df):
    credit_history_in_months = np.round((lendingClub_df["Issued Date"] - lendingClub_df["Earliest Credit Line"])/np.timedelta64(1,'M'))
    lendingClub_df["Credit History Length in Months"] = credit_history_in_months
    return lendingClub_df

# Round the Total Received Principle.
def round_rec_principle(lendingClub_df):
    lendingClub_df["Total Received Principle"] = np.round(lendingClub_df["Total Received Principle"])
    return lendingClub_df

# Present the interest rates in %.
def scale_interest_rate(lendingClub_df):
    lendingClub_df["Interest Rate"] *= 100
    return lendingClub_df

# training_nonempty.pipe(format_description).pipe(add_credit_history_in_months).pipe(round_rec_principle).pipe(scale_interest_rate)
# test_nonempty.pipe(format_description).pipe(add_credit_history_in_months).pipe(round_rec_principle).pipe(scale_interest_rate)

# # Check if Derogatory Public Records contain the same information as Public Bankcruptcy Records
# (training_nonempty["Derogatory Public Records"] != training_nonempty["Public Bankcruptcy Records"]).sum()
# # Check if Derogatory Public Records also contain bankcruptcy.
# (training_nonempty["Derogatory Public Records"] < training_nonempty["Public Bankcruptcy Records"]).sum()

### 1.5 Expanding Data Scope

The traditional credit scoring model doesn't explicitly take into account macro-economic conditions; machine learning models, due to their flexibility in input, should easily improve on that. However our dataset doesn't contain any macro-economic variables. Utilizing free public resources, we introduced macro-economic variables with different granularity:

- Monthly GDP growth: national level. GDP growth reflects the health in overall economy, which should be negatively correlated with default rate. We expect this variable to capture economy-level information, leaving more detailed information in geography to other variables.
- Monthly unemployment: state and zip-code level. One's prospect of unemployment is more intimately affected by his immediate environment, so we use zip-code level (a three-digit zipcode typically includes multiple counties) data. We also include state-level data because for some zip-codes the data are missing. 
- Per capita income: zip-code level. We will use this data to check against the unverified incomes.
- Median housing price and rent: zip-code level. These should further differentiate the impact of homeownership. 

__Adding Unemployment Data__

We consulted the Bereau of Labor Statistics website and downloaded the montly unemployment data for each state. We then added to each loan application the statewide unemployment rate at the time of issuance.

In [15]:
%%time
## Add state-specfic unemployment rate during the issuance month
unemployment = pd.read_csv("../data/unemployment.csv")
unemployment.reset_index(drop=True, inplace=True)
unemployment["date"] = pd.date_range(start='2006-01-01', end='2015-12-01', freq="MS")
unemployment.set_index("date", inplace=True)


def get_unemployment(row):
    issued_date = row['Issued Date']
    state = row['State']
    return unemployment.loc[issued_date, state]

def add_unemployment(lendingClub_df):
    unemployment = []
    for i in range(lendingClub_df.shape[0]):
        unemployment.append(get_unemployment(lendingClub_df.iloc[i, :]))
    unemployment = np.array(unemployment)
    lendingClub_df["Statewide Unemployment at Issuance"] = unemployment
    return lendingClub_df

# test_nonempty.pipe(add_unemployment)
# training_nonempty.pipe(add_unemployment)

Wall time: 15 ms


__Adding Quarterly GDP Growth__

We obtained our data from St.Lious Fed https://fred.stlouisfed.org/series/A191RL1Q225SBEA.

In [16]:
gdp = pd.read_csv("../data/us_quarterly_gdp_growth.csv")
dates = pd.to_datetime(gdp["DATE"])
gdp["Year"] = dates.dt.year
gdp["Quarter"] = dates.dt.quarter
gdp.drop(columns = ["DATE"], inplace=True)
gdp.columns = ["GDP Quarterly Growth at Issuance", "Year", "Quarter"]

def add_gdp(lendingClub_df):
    lendingClub_df["Year"] = lendingClub_df["Issued Date"].dt.year
    lendingClub_df["Quarter"] = lendingClub_df["Issued Date"].dt.quarter
    temp = lendingClub_df.merge(gdp, on=["Year", "Quarter"], how='left')
    lendingClub_df["GDP Quarterly Growth at Issuance"] = temp["GDP Quarterly Growth at Issuance"]
    lendingClub_df.drop(columns=["Year", "Quarter"], inplace=True)
    return lendingClub_df

# training_nonempty.pipe(add_gdp)
# test_nonempty.pipe(add_gdp)

__Inferring Industry Section from Employer Title: Too Many Missing Values__

We consulted a NASDAQ database (https://public.opendatasoft.com/explore/dataset/us-companies-names-industries/export/) for mapping from company name to industry, and added each employer's industry based on a `Employer Title` name search. 

The result is less than ideal -- out more than 35000 samples in the training set, only 4800+ entries found their industry. Is this because there aren't that many applicants who work for publicly listed companies? According to a WSJ estimate(https://www.nysscpa.org/news/publications/the-trusted-professional/article/more-americans-work-at-big-firms-than-small-ones-040717), in 2014, small companies (used as a proxy for private companies) employ about 1.5 times as many people as large companies. 

The distortion in our data is clearly larger, which suggests our text processing on `Employer Title` might have been too coarse. For this project, we won't include the industry section data; <span style=color:red> for future iterations, we recommend doing keyword extraction on Employer Title before proceeding to search. </span>

In [17]:
## Adding industry according to company title
company_industry = pd.read_excel("../data/us_company_names_industries.xlsx")
company_industry.columns = ["Name", "Industry"]

In [18]:
# Remove punctuations and NAs from Employer Title
def format_emp_title(lendingClub_df):
    mask = (lendingClub_df["Employer Title"] == "")
    (lendingClub_df["Employer Title"])[mask] = "NOT PROVIDED"
    lendingClub_df["Employer Title"] = lendingClub_df["Employer Title"].str.replace('[{}]'.format(string.punctuation), "")
    lendingClub_df["Employer Title"].fillna("NOT PROVIDED", inplace=True)
    return lendingClub_df

# training_nonempty.pipe(format_emp_title)
# test_nonempty.pipe(format_emp_title)

In [19]:
%%time
## This chunk will take approximately 13 minutes...
def index_of_first_true(lst):
    for i, v in enumerate(lst):
        if v is True:
            return i
    return -1

def get_industry(row):
    emp_title = row["Employer Title"]
    bools = company_industry["Name"].str.contains(emp_title, case=False)
    first_true = index_of_first_true(bools)
    if first_true == -1:
        return "Not Listed"
    else:
        return company_industry.loc[first_true, "Industry"]

def add_industry(lendingClub_df):
    industry = []
    for i in range(lendingClub_df.shape[0]):
        industry.append(get_industry(lendingClub_df.iloc[i, :]))
    industry = np.array(industry)
    lendingClub_df["Industry"] = industry
    return lendingClub_df

# test_nonempty.pipe(add_industry)
# training_nonempty.pipe(add_industry)

Wall time: 0 ns


#### Adding Zip-code based Income, Rent, Housing Price and Unemployment.

The data provided by the census has the geographical categories as county codes. However Lending club data has only three digit zip code. We downloaded a mapping from Missouri Census Data Center to map three digit zip codes to county codes.

http://mcdc.missouri.edu/applications/geocorr2014.html

We know that for each county code, there will be an associated three digit zip code. However there might be more than one county code for each three digit zip code. Therefore we create a dictionary to assign a 3 digit zip code for each county code, which will be used to calculate 3 digit zip code level census data.

In [20]:
#Read the Missouri Data Science Center Zip Code County Code Map file
zip_county_map = pd.read_csv("../data/census_data/zip_census_tract.csv",encoding="latin-1",dtype=str)

#necessary columns
zip_county_map_columns =['ZIP census tabulation area', 'County code',"Tract"]
zip_county_map["Tract"]=zip_county_map["Tract"].str.replace('.',"")
zip_county_map["County Tract Code"]=zip_county_map["County code"]+zip_county_map["Tract"]
zip_county_map_columns =['ZIP census tabulation area', 'County Tract Code']
zip_county_map_drop_cols=[x for x in zip_county_map if x not in zip_county_map_columns]

#drop unnecessary columns
zip_county_map.drop(columns=zip_county_map_drop_cols, axis=0,inplace=True)


#calculate 3 digit zip code
zip_county_map["3 Digit Zip"]= [x[:3] if len(x)==5 else x[:2] for x in zip_county_map['ZIP census tabulation area']]
county_zip_map=dict(zip(zip_county_map["County Tract Code"],zip_county_map["3 Digit Zip"]))

Per capita income, median house value, median gross rent, and unemployed population over 16 years old data is obtained from the American Cummunity Survey.

We download the dictionary for the variable names in the data. Strip white spaces and store in a dictionary file

In [21]:
#Read the American Community Survey Dictionary file
census_dict = pd.read_csv("../data/census_data/census_dictionary.csv")

#remove extra columns that will not be used
census_dict.drop(columns=[x for x in census_dict.columns if x not in ["Code","Description"]],inplace=True)

#rename the column name so that the whitespace is trimmed.
census_dict.columns=census_dict.columns.str.lstrip().str.rstrip()

#remove whitespace before and after string
census_dict["Code"]=census_dict["Code"].str.lstrip().str.rstrip()
# census_dict["FIPS"]=census_dict["FIPS"].str.rstrip(" ")

census_code_dict = dict(zip(census_dict["Code"],census_dict["Description"]))
census_description_dict = dict(zip(census_dict["Description"],census_dict["Code"]))

For the per capita income, Total Population over 16 Years Old, Total Population variables, we will simply sum each county code in the 3 digit zip code level.

However, for median house value and median gross rent, the underlying data is not available. Therefore although it is not absolutely correct, we average these variables to get average median house value and average median gross rent for each three digit zip code.

Below is the weighted average function to be applied.

In [22]:
def weighted_average_funct(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()
    


Read the American Census data csv file, specifying the data type for the variables that will be used.

In [23]:
#read the American Community Survey (2006-2010) data.
#Data is about 67mb.
#source is Social Explorer, granted a student licence through Rutgers University.
#encoding of the source file is Western Latin-1 (ISO-8859-1)

census_data = pd.read_csv("../data/census_data/ACS_2006_2010_census_tract.csv",encoding ="latin-1",
                         dtype={"Geo_FIPS": str,
                                census_description_dict['     Total Population    ']: np.int64,
                                census_description_dict['     Civilian Population in Labor Force 16 Years and Over    ']: np.int64,
                                census_description_dict['     Civilian Population in Labor Force 16 Years and Over  Unemployed   ']: np.int64,
                                census_description_dict['     Per Capita Income (In 2010 Inflation Adjusted Dollars)    ']: np.float64,
                                census_description_dict['     Median Value    ']: np.float64,
                                census_description_dict['     Median Gross Rent    ']: np.float64,})

Below we clean out the column names, change the variable codes to descriptions, drop unnecessary columns.

In [24]:

#strip the variable categorical indicators and underscores
census_data.columns=census_data.columns.str.lstrip("Geo")
census_data.columns=census_data.columns.str.lstrip("SE")
census_data.columns=census_data.columns.str.lstrip("_")

#replace variable codes with variable names
census_data.columns=[x if x not in census_code_dict else census_code_dict[x] for x in census_data.columns]

#columns to keep from the data source, we will drop every other column
census_data_columns_keep = ['FIPS',
                        '     Total Population    ',
                       '     Civilian Population in Labor Force 16 Years and Over    ',
                       '     Civilian Population in Labor Force 16 Years and Over  Unemployed   ',
                       '     Per Capita Income (In 2010 Inflation Adjusted Dollars)    ',
                       '     Median Value    ',
                        '     Median Gross Rent    ']

#columns to remove
census_data_columns_remove=[x for x in census_data.columns if x not in census_data_columns_keep]

#drop columns
census_data.drop(columns=census_data_columns_remove,inplace=True)

#strip whitespace from end and beginning
census_data.columns=census_data.columns.str.lstrip().str.rstrip()

#rename the column names
census_data_columns_newnames = ['County Tract Code',
                                'Total Population',
                                'Total Population over 16 Years Old',
                                'Unemployed',
                                'Per Capita Income',
                                'Median House Value', 
                                'Median Gross Rent']

census_data.columns = census_data_columns_newnames


We now map the the zip code information in the census data for each county code.

In [25]:
census_data["County Tract Code"]=census_data["County Tract Code"].astype(str)
census_data["3 Digit Zip Code"]=census_data['County Tract Code'].map(county_zip_map)

#number of unique values for 3 digit zip code in the census data
census_data["3 Digit Zip Code"].nunique()

889

We now calculate the weighted average of the four county wide data to obtaine zip code level data for median house value and gross rent simply sum other variables.

We also calculate the unemployment rate from the unemployed population data and we combine everything in the original dataframe

In [26]:
#We now calculate the weighted average of the four county wide data to obtaine zip code level data for median house value and gross rent
census_group_house_value= census_data.groupby("3 Digit Zip Code").apply(weighted_average_funct,"Median House Value","Total Population").reset_index(name='Average Median House Value')
census_group_rent= census_data.groupby("3 Digit Zip Code").apply(weighted_average_funct,"Median Gross Rent","Total Population").reset_index(name='Average Median Gross Rent')
census_group_per_capita= census_data.groupby("3 Digit Zip Code").apply(weighted_average_funct,"Per Capita Income","Total Population").reset_index(name='Per Capita Income')

f = {'Total Population': sum, 'Total Population over 16 Years Old': sum,"Unemployed": sum}
census_group_main= census_data.groupby("3 Digit Zip Code").agg(f)

#calculate simple unemployment statistics
census_group_main["Unemployment Rate"]=census_group_main["Unemployed"]/census_group_main["Total Population over 16 Years Old"]

census_group_main.reset_index(inplace=True)

census_group_main.drop(columns=["Total Population over 16 Years Old"],inplace=True)


census_group_main["Average Median House Value"]= census_group_house_value["Average Median House Value"]
census_group_main["Average Median Gross Rent"]= census_group_rent["Average Median Gross Rent"]
census_group_main["Per Capita Income"]= census_group_per_capita["Per Capita Income"]


# census_group_main["Average Median Gross Rent"]=census_group_house_value

census_data=census_group_main

census_data.head()



,3 Digit Zip Code,Total Population,Unemployed,Unemployment Rate,Average Median House Value,Average Median Gross Rent,Per Capita Income
0,010,450318,18277,0.076412,226278.198740,765.872874,27445.965298
1,011,168665,9866,0.130560,176337.901758,804.549895,20864.963187
2,012,131116,5137,0.075136,219597.476281,745.439443,28301.380426
3,013,81116,3216,0.071128,217177.670250,828.543296,27044.392808
4,014,204887,8396,0.076749,274537.244432,877.998448,29036.776599


In [32]:
# unemployment_dict=dict(zip(census_data['County Code'],census_data["Unemployment Rate"]))
per_capita_income_dict=dict(zip(census_data['3 Digit Zip Code'],census_data["Per Capita Income"]))
total_population_dict=dict(zip(census_data['3 Digit Zip Code'],census_data["Total Population"]))
average_median_house_value_dict=dict(zip(census_data['3 Digit Zip Code'],census_data["Average Median House Value"]))
average_median_rent_dict=dict(zip(census_data['3 Digit Zip Code'],census_data["Average Median Gross Rent"]))
unemployment_dict=dict(zip(census_data['3 Digit Zip Code'],census_data["Unemployment Rate"]))


def add_county_data(lendingclub_df):
    
    #first create a three digit zip code from the 5 digit code.
    lendingclub_df["3 Digit Zip Code"]=lendingclub_df["Zip Code"].str.rstrip("xx").str.lstrip("0")
    
    #now add census data
    lendingclub_df["Unemployment Rate"]=lendingclub_df["3 Digit Zip Code"].map(unemployment_dict)
    lendingclub_df["Unemployment Rate"] *= 100
    lendingclub_df["Average Median Gross Rent"]=lendingclub_df["3 Digit Zip Code"].map(average_median_rent_dict)
    lendingclub_df["Average Median House Value"]=lendingclub_df["3 Digit Zip Code"].map(average_median_house_value_dict)
    lendingclub_df["Total Population"]=lendingclub_df["3 Digit Zip Code"].map(total_population_dict)
    lendingclub_df["Per Capita Income"]=lendingclub_df["3 Digit Zip Code"].map(per_capita_income_dict)

    
    return lendingclub_df


### 1.6 A Note on Automated Data Cleaning

For any future iterations on new data, run the notebook once, and run the cells below.

A couple of caveats: 
- the new dataset has to have exactly the same columns where there's no variation. This is because if the dataset contains too few data points, the `Issued Date` for all entries will be the same, and we will have an extra column removed in the `drop_columns_no_variation` step. 
- The external data on unemployment and gdp ranges from the beginning of 2006 to the end of 2015. Further, the raw unemployment data are manually downloaded from BLS website. If user wants to reference periods beyond this range, he/she has to manually update the raw data. 

As a rule of thumb for using the current iteration, make sure the new dataset contains at least 2000 entries.

In [28]:
def get_non_empty_columns(lendingClub_df_raw):
    training_nonempty = lendingClub_df_raw[list(non_empty_columns(rawTraining))]
    # Format Variable Names.
    training_nonempty.columns = variable_name_tidy
    training_nonempty = training_nonempty.reindex(columns=variable_names_grouped_list)
    return training_nonempty

def missing_value_handling(lendingClub_df):
    lendingClub_df.pipe(replace_with_empty_string).pipe(fill_blank_months_with_zero).pipe(replace_empty_last_pmt_d_with_issuance_d).pipe(drop_no_data_entries);
    return lendingClub_df

def variable_transformation(lendingClub_df):
    lendingClub_df.pipe(add_credit_history_in_months).pipe(round_rec_principle).pipe(scale_interest_rate).pipe(drop_columns_no_variation).pipe(chargeoff_indicator).pipe(format_description)
    return lendingClub_df

def add_external_data(lendingClub_df):
    lendingClub_df.pipe(format_emp_title).pipe(add_unemployment).pipe(add_gdp).pipe(add_county_data)
    return lendingClub_df
    
def clean_data(lendingClub_df_raw):
    lendingClub_df_raw = lendingClub_df_raw.pipe(get_non_empty_columns)
    lendingClub_df_raw.pipe(missing_value_handling).pipe(variable_transformation).pipe(add_external_data)
    return lendingClub_df_raw

In [33]:
%%time
training = rawTraining.pipe(clean_data)
test = rawTest.pipe(clean_data)

C:\Users\nleea\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nleea\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Wall time: 13.4 s


In [35]:
training.head(1)

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Installment,Grade,Sub Grade,Loan Status,Issued Date,Title,Description,Purpose,Employer Title,Employment Length,Home Ownership,Annual Income,Verification Status,Zip Code,State,Debt-to-income Ratio,Delinquencies in 2 years,Earliest Credit Line,Inquiries in 6 months,Months Since Last Delinquency,Months Since Last Public Record,Open Accounts,Derogatory Public Records,Revolving Balance,Revolving Balance Utilization,Number of Total Accounts,Last Credit Pulled Date,Public Bankcruptcy Records,Total Payment,Total Payment Investor,Total Received Principle,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Last Payment Date,Last Payment Amount,Debt Settlement Flag,Debt Settlement Flag Date,Settlement Status,Settlement Date,Settlement Amount,Settlement Percentage,Settlement Term,Credit History Length in Months,Statewide Unemployment at Issuance,GDP Quarterly Growth at Issuance,3 Digit Zip Code,Unemployment Rate,Average Median Gross Rent,Average Median House Value,Total Population,Per Capita Income
0,35000,35000,34975.0,60 months,11.71,773.44,B,B3,1,2011-11-01,Restaurant Inventory,Loan proceeds will be used to partially fund a...,small_business,US Department of Labor,10+ years,MORTGAGE,110000.0,Verified,945xx,CA,1.06,0,1971-01-01,0,0.0,0.0,10,0,4142,0.064,27,2017-07-01,0.0,11601.6,11593.34,6927.0,4652.28,0.0,22.5,0.0,2013-02-01,773.44,N,NaT,NaN,NaT,NaN,NaN,NaN,490.0,11.3,4.7,945,8.176195,1425.191917,546049.175985,2025070.0,35498.327248


In [ ]:
%%time
training.to_excel("../data/training_cleaned.xlsx", index=False)
test.to_excel("../data/test_cleaned.xlsx", index=False)